In [ ]:
data=[]

In [ ]:
data.append(
    {
        "image": "C:/AI/Github/Reconnaissance_drone_report/Data/Images/earthquake/download.jpg",
        "text": "Describe the image.",
        "content": "This image shows a distructed building due to earthquake since there is a lot of debris is setteled in this area"
    }
)

In [2]:

import os
import json
import torch
from PIL import Image
from transformers import (
    Qwen2VLForConditionalGeneration,
    Qwen2VLProcessor,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


In [4]:

MODEL_NAME = "Qwen/Qwen2-VL"
DATASET_PATH = "C:/AI/Github/Reconnaissance_drone_report/train_dataset.json"
IMAGE_DIR = "C:/AI/Github/Reconnaissance_drone_report/Data/Images"
OUTPUT_DIR = "./qwen2-vl-lora-finetuned"


In [5]:

def create_dataset(json_path, image_dir):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = []
    for item in data:
        image_path = os.path.join(image_dir, item["image"])
        if os.path.exists(image_path):
            processed_data.append({
                "image": Image.open(image_path).convert("RGB"),
                "query": item["query"],
                "description": item["description"]
            })
    return processed_data


In [6]:

def collator(features):
    processor = Qwen2VLProcessor.from_pretrained(MODEL_NAME)
    images = [f["image"] for f in features]
    queries = [f["query"] for f in features]
    descriptions = [f["description"] for f in features]

    inputs = processor(images=images, text=queries, return_tensors="pt", padding=True, truncation=True)
    labels = processor.tokenizer(descriptions, return_tensors="pt", padding=True, truncation=True).input_ids
    inputs["labels"] = labels
    return inputs


In [ ]:

from transformers import BitsAndBytesConfig

#bnb_config = BitsAndBytesConfig(
#    load_in_8bit=True,
#    llm_int8_enable_fp32_cpu_offload=True
#)

#model = Qwen2VLForConditionalGeneration.from_pretrained(
#    MODEL_NAME,
#    quantization_config=bnb_config,
#    device_map="auto",
#    torch_dtype=torch.float16,
#    trust_remote_code=True
#)
from transformers import Qwen2VLForConditionalGeneration

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen-VL",
    torch_dtype=torch.float16,
    device_map={"": 0},  # Force entire model on GPU 0 (your 4060)
    trust_remote_code=True
)


model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


You are using a model of type qwen to instantiate a model of type qwen2_vl. This is not supported for all configurations of models and can yield errors.


In [ ]:

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    remove_unused_columns=False,
    label_names=["labels"]
)

dataset = create_dataset(DATASET_PATH, IMAGE_DIR)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=collator
)


In [ ]:
trainer.train()
model.save_pretrained(OUTPUT_DIR)
Qwen2VLProcessor.from_pretrained(MODEL_NAME).save_pretrained(OUTPUT_DIR)
print("Training complete! Model saved to:", OUTPUT_DIR)

In [3]:
torch.cuda.empty_cache()

In [ ]:
# train_reconnaissance_drone.py

import os
import json
import torch
from PIL import Image
from transformers import (
    Qwen2VLForConditionalGeneration,
    Qwen2VLProcessor,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset

# Configuration
MODEL_NAME = "Qwen/Qwen2-VL-2B-Instruct"
DATASET_PATH = "train_dataset.json"  # Dataset is directly in root
IMAGE_DIR = "Data/Images"            # Images are in Data/Images
OUTPUT_DIR = "ayntb_checkpoints"       # Checkpoints are in ayntb_checkpoints
LORA_CONFIG = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 1. Prepare Dataset
def create_dataset(json_path, image_dir):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = []
    for item in data:
        image_path = item.get("image")  # Get image path
        query = item.get("query")  # Get query
        description = item.get("description")  # Get description

        if not image_path or not query or not description:
            print(f"Warning: Skipping item due to missing data: {item}")
            continue

        try:
            # Load the image
            if not os.path.isabs(image_path):
              image_path = os.path.join(image_dir, image_path) # Use image_dir
            img = Image.open(image_path)
            img = img.convert("RGB")  # Ensure consistent image format
            processed_data.append({
                "image": img,
                "query": query,
                "description": description
            })
        except FileNotFoundError:
            print(f"Warning: Image not found at {image_path}")
            continue
        except Exception as e:
            print(f"Error loading image: {e}")
            continue

    if not processed_data:
        print("Error: No valid data found in the dataset. Check your JSON file and image paths.")
        return None  # or raise an exception

    return Dataset.from_list(processed_data)


# 2. Data Collator
def collator(features):
    processor = Qwen2VLProcessor.from_pretrained(MODEL_NAME)

    images = [feature["image"] for feature in features]
    queries = [feature["query"] for feature in features]
    descriptions = [feature["description"] for feature in features]

    inputs = processor(images=images, text=queries, return_tensors="pt", padding=True, truncation=True)

    # Prepare labels (descriptions)
    labels = processor.tokenizer(descriptions, return_tensors="pt", padding=True, truncation=True).input_ids
    inputs["labels"] = labels  # Add labels to inputs
    return inputs


# 3. Load Model and Processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # Keep this for mixed precision
)
processor = Qwen2VLProcessor.from_pretrained(MODEL_NAME)

# Apply LoRA
model = get_peft_model(model, LORA_CONFIG)
model.print_trainable_parameters()

# 4. Training Setup
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,  # Keep this for mixed precision
    save_strategy="epoch",
    logging_steps=10,
    remove_unused_columns=False,
    optim="adafactor",  # Memory-efficient optimizer
    report_to="none"     # Disable TensorBoard/WandB
)

# 5. Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=create_dataset(DATASET_PATH, IMAGE_DIR),
    data_collator=collator,
)

# 6. Start Training
trainer.train()
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

print("Training complete! Model saved to:", OUTPUT_DIR)
